In [4]:
import getpass
import os
password = getpass.getpass()
command = "sudo -S ap update" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))
command = "sudo -S apt install -y build-essential g++ gcc make ninja-build" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))

os.environ["CUDA_HOME"] = "/usr/local/cuda-12.4"
os.environ["LD_LIBRARY_PATH"] = (
    "/usr/lib/x86_64-linux-gnu:/usr/local/cuda-12.4/lib64:"
    + os.environ.get("LD_LIBRARY_PATH", "")
)

[sudo] password for user: sudo: ap: command not found
[sudo] password for user: 



Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.10ubuntu1).
g++ is already the newest version (4:13.2.0-7ubuntu1).
gcc is already the newest version (4:13.2.0-7ubuntu1).
make is already the newest version (4.3-4.1build2).
ninja-build is already the newest version (1.11.1-2).
0 upgraded, 0 newly installed, 0 to remove and 247 not upgraded.


In [5]:
# export_reg_gated_vit.py

import torch
from transformers import CLIPVisionConfig, CLIPVisionModel
from INFERclipregXGATED.model import VisionTransformer

# 1) 커스텀 비전 트랜스포머 인스턴스 생성 & 체크포인트 로드
VISION_ENCODER_CONFIG = dict(
    image_resolution=224, patch_size=14,
    width=1024, layers=24, heads=16,
    output_dim=1024, num_registers=4
)
custom_vt = VisionTransformer(
    input_resolution=VISION_ENCODER_CONFIG['image_resolution'],
    patch_size=      VISION_ENCODER_CONFIG['patch_size'],
    width=           VISION_ENCODER_CONFIG['width'],
    layers=          VISION_ENCODER_CONFIG['layers'],
    heads=           VISION_ENCODER_CONFIG['heads'],
    output_dim=      VISION_ENCODER_CONFIG['output_dim'],
    num_registers=   VISION_ENCODER_CONFIG['num_registers'],
)

In [3]:
# (예: safetensors 사용 시)
state = torch.load("models/ViT-L-14-REG-GATED-balanced-ckpt12.safetensors", map_location="cpu")
# 여기서는 state_dict 키에서 "visual." prefix 제거
vt_state = {k.split("visual.",1)[-1]:v for k,v in state.items() if k.startswith("visual.")}
custom_vt.load_state_dict(vt_state, strict=False)

UnpicklingError: invalid load key, '`'.

In [ ]:
# 2) HF CLIPVisionModel 구성 같은 아키텍처로 생성
hf_cfg = CLIPVisionConfig(
    image_size=224,
    patch_size=14,
    hidden_size=1024,
    num_hidden_layers=24,
    num_attention_heads=16,
    intermediate_size=1024 * 4,   # typically 4× hidden_size
)
hf_model = CLIPVisionModel(hf_cfg)

In [ ]:
# 3) 커스텀 가중치를 HF 모델에 복사
#    -- custom_vt.state_dict()의 키와 hf_model.visual.state_dict()의 키가 일치해야 합니다.
hf_state = hf_model.visual.state_dict()
for name, tensor in custom_vt.state_dict().items():
    if name in hf_state and hf_state[name].shape == tensor.shape:
        hf_state[name] = tensor
hf_model.visual.load_state_dict(hf_state, strict=True)

In [ ]:
# 4) HF 구조로 저장
hf_model.save_pretrained("./reg_gated_vit_l14")
print("✅ Saved Reg-Gated ViT-L/14 as HF format in ./reg_gated_vit_l14")
